# Step 1 Install all python libraries

In [ ]:
!python -m pip install semantic-kernel
!python -m pip install azure.monitor
!python -m pip install pandas
!python -m pip install azure.identity
!python -m pip install azure.core


# Step 2 Import Packages required Prepare a semantic kernel instance first


In [ ]:
from dotenv import dotenv_values
import semantic_kernel as sk
from semantic_kernel import ContextVariables # Context to store variables and Kernel to interact with the kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion ,AzureTextEmbedding # AI services
import pandas as pd


kernel = sk.Kernel() # Create a kernel instance
kernel1 = sk.Kernel() #create another kernel instance for not having semanitc function in the same kernel 

useAzureOpenAI = True
config = dotenv_values(".env")
AZURE_OPENAI_API_KEY = config["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = config["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel1.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

# Step 3 Importing skills and function from folder

In [ ]:
plugins_directory = "./plugins"
DevFunctions=kernel1.import_semantic_skill_from_directory(plugins_directory, "AzureMonitor") # import the skill from the folder
FDesFunction = DevFunctions["KQLquerySignin"]  # get the semantic function

# Step 4 calling the semantic function with string

In [ ]:
  
resultFD = FDesFunction("please share all sign-in location?")
print(resultFD)


# Step 5 Importing native function

In [ ]:
from plugins.AzureMonitor.native_function import sloganalytics
Mon_plugin = kernel.import_skill(sloganalytics(kernel1), skill_name="AzureMonitor") # import the skill
variables = ContextVariables()
variables["description"] = "test"

# Step 6 Executing native function

In [ ]:
variables["title"] = "please share all sign-in location?"
result = await kernel.run_async( Mon_plugin["slquery"], input_vars=variables )
print(variables["Df"])